# Paper Figure Creation

- Created on a cloudly London Saturday morning, April 3rd 2021
- Revised versions of the figures

In [19]:
import climlab
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import xarray as xr
import pandas as pd
import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from IPython.display import clear_output
import time
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.patches as patches
from matplotlib.colors import LogNorm
import matplotlib.colors
import matplotlib as mpl

# Fig. 1

In [20]:
values = xr.open_dataset('../../Data/CERES/clear_sky_ceres.nc')
landmask = xr.open_dataset('../../Data/Other/landsea.nc')

lats = values.lat.values
lons = values.lon.values

In [ ]:
# Variables that you want to plot
plotvar1 = values.r2.values

# Adding a cyclic point to the two variables
# This removes a white line at lon = 0
lon_long = values.lon.values
plotvar_cyc1 = np.zeros((len(lats), len(lons)))
plotvar_cyc1, lon_long = add_cyclic_point(plotvar_cyc1, coord=lon_long)
for i in range(len(lats)):
    for j in range(len(lons)):
        plotvar_cyc1[i, j] = plotvar1[i, j]
plotvar_cyc1[:, len(lons)] = plotvar_cyc1[:, 0]

# Plotting
fig = plt.figure(figsize=(6, 2.7), constrained_layout=True)
width_vals = [2, 1]
gs = fig.add_gridspec(ncols=2, nrows=1, width_ratios=width_vals)

SIZE = 8
plt.rc('font', size=SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=SIZE)     # fontsize of the x and y labels
plt.rc('xtick', labelsize=SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SIZE)    # legend fontsize
plt.rc('figure', titlesize=SIZE)   # fontsize of the figure title

# Upper left map
ax1 = fig.add_subplot(gs[0], projection=ccrs.PlateCarree())
ax1.coastlines()
ax1.set_title("a) Map of R$^2$ for Linear Fit of Monthly OLR to Monthly T$_S$")
C1 = ax1.pcolor(
    lon_long, lats, plotvar_cyc1, transform=ccrs.PlateCarree(), cmap='RdYlGn', rasterized=True
)

ax1.set_xticks([-180, -90, 0, 90, 180], crs=ccrs.PlateCarree())
ax1.set_yticks([-90, 0, 90], crs=ccrs.PlateCarree())
lon_formatter = LongitudeFormatter(number_format='.0f',
                                   dateline_direction_label=True)
lat_formatter = LatitudeFormatter(number_format='.0f')
ax1.xaxis.set_major_formatter(lon_formatter)
ax1.yaxis.set_major_formatter(lat_formatter)


# Colourbars
cbar = fig.colorbar(
    C1,
    ax=ax1,
    label=r"$R^2$",
    fraction=0.1,
    orientation="horizontal",
    ticks=[0.001, 0.2, 0.4, 0.6, 0.8, 0.999]
)

cbar.ax.set_xticklabels(['0', '0.2', '0.4', '0.6', '0.8', '1'])

ax1.text(110, 70, 'a',    horizontalalignment='center', verticalalignment='center', color='white',
         fontsize=8, fontweight='bold', bbox={'facecolor': 'black', 'edgecolor': 'none', 'alpha': 0.5, 'pad': 5})
ax1.text(-110, -65, 'b', horizontalalignment='center', verticalalignment='center', color='white',
         fontsize=8, fontweight='bold', bbox={'facecolor': 'black', 'edgecolor': 'none', 'alpha': 0.5, 'pad': 5})
ax1.text(27, -2, 'c',    horizontalalignment='center', verticalalignment='center', color='white',
         fontsize=8, fontweight='bold', bbox={'facecolor': 'black', 'edgecolor': 'none', 'alpha': 0.5, 'pad': 5})
ax1.text(0, -10, 'd',    horizontalalignment='center', verticalalignment='center', color='white',
         fontsize=8, fontweight='bold', bbox={'facecolor': 'black', 'edgecolor': 'none', 'alpha': 0.5, 'pad': 5})
ax1.text(80, 15, 'e',    horizontalalignment='center', verticalalignment='center', color='white',
         fontsize=8, fontweight='bold', bbox={'facecolor': 'black', 'edgecolor': 'none', 'alpha': 0.5, 'pad': 5})

# Upper right map
ax2 = fig.add_subplot(gs[1])

extra_tropics = list(np.arange(-90, -29, 1))
extra_tropics += list(np.arange(30, 91, 1))

tropics = list(np.arange(-30, 31, 1))

mask_sea_t = landmask.interp_like(values, method='nearest').sel(
    lat=tropics).LSMASK.values == 0
mask_land_t = landmask.interp_like(
    values, method='nearest').sel(lat=tropics).LSMASK.values == 1
mask_sea_et = landmask.interp_like(values, method='nearest').sel(
    lat=extra_tropics).LSMASK.values == 0
mask_land_et = landmask.interp_like(values, method='nearest').sel(
    lat=extra_tropics).LSMASK.values == 1

bnum = np.arange(-3, 5, 0.2)
lw = 2

ax2.hist(values.sel(lat=extra_tropics).grad.values[mask_land_et].flatten(
), bins=bnum, density=True, histtype='step', linewidth=lw, label='Extratropics:\nLand', color='C1')
ax2.hist(values.sel(lat=extra_tropics).grad.values[mask_sea_et].flatten(
),  bins=bnum, density=True, histtype='step', linewidth=lw, label='Extratropics:\nOcean', color='C0')
ax2.hist(values.sel(lat=tropics).grad.values[mask_land_t].flatten(
), bins=bnum, density=True, histtype='step', linewidth=lw, label='Tropics:\nLand', color='red')
ax2.hist(values.sel(lat=tropics).grad.values[mask_sea_t].flatten(
),  bins=bnum, density=True, histtype='step', linewidth=lw, label='Tropics:\nOcean', color='navy')

ax2.set_xlim(-3, 5)
ax2.set_title("b) Histogram of Slope $\partial$OLR/$\partial$T$_S$")
ax2.set_xlabel(r'Linear Slope $\partial$OLR/$\partial$T$_S$ (Wm$^{-2}$/K)')
ax2.set_ylabel('Probability Density')
ax2.legend(loc='upper left', handlelength=0.1)

ax1.set_anchor('N')
ax2.set_anchor('N')

path = "../../Figures/After first review/"
plt.savefig(path + 'Fig 1 CERES NEW.pdf',
            bbox_inches='tight', dpi=300)
plt.close()

### Calculating Mean Values for the Figure Caption

In [21]:
area_factors = np.zeros_like(values.mean(dim='month').t2m.values)

lats = values.lat.values
lons = values.lon.values

for i in range(len(lats)):
    area_factors[i, :] = np.cos(lats[i]*2*np.pi/360)

values['area_factors'] = (('lat', 'lon'), area_factors)

grad_area_scaled = np.zeros_like(values.mean(dim='month').t2m.values)

for i in range(len(lats)):
    for j in range(len(lons)):
        grad_area_scaled[i,j] = values.sel(lat=lats[i], lon=lons[j]).area_factors.values[()] * values.sel(lat=lats[i], lon=lons[j]).grad.values[()]

values['grad_area_scaled'] = (('lat', 'lon'), grad_area_scaled)

glbl = np.sum(values.grad_area_scaled.values.flatten()) / np.sum(values.area_factors.values.flatten())

print('Global area weighted mean is', glbl)

Global area weighted mean is 0.9909934908510208


# Fig. 2

In [3]:
values_meas = xr.open_dataset('../../Data/CERES/clear_sky_ceres.nc')

lats = values_meas.lat.values
lons = values_meas.lon.values

In [ ]:
fig = plt.figure(figsize=(6, 4.6), constrained_layout=True)
height_vals = [1, 4]
gs = fig.add_gridspec(ncols=5, nrows=2, height_ratios=height_vals)

month_list = np.arange(1, 13)
lc1 = "#7d7d7d"

cvals = [1., 2., 3., 4., 5., 6., 7., 8., 9., 10., 11., 12.]
colors = ["#f1423f", "#f1613e", "#f79a33", "#feba28", "#efe720", "#b6d434",
          "#00b34e", "#0098d1", "#0365b0", "#3e3f9b", "#83459b", "#bd2755"]

norm = plt.Normalize(min(cvals), max(cvals))
tuples = list(zip(map(norm, cvals), colors))
cmap_new = matplotlib.colors.LinearSegmentedColormap.from_list("", tuples)

SIZE = 8
plt.rc('font', size=SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=SIZE)     # fontsize of the x and y labels
plt.rc('xtick', labelsize=SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SIZE)    # legend fontsize
plt.rc('figure', titlesize=SIZE)   # fontsize of the figure title

ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[0, 1])
ax3 = fig.add_subplot(gs[0, 2])
ax4 = fig.add_subplot(gs[0, 3])
ax5 = fig.add_subplot(gs[0, 4])

top_axs = [ax1, ax2, ax3, ax4, ax5]

axs = fig.add_subplot(gs[1, :])

axs.scatter(
    values_meas.sel(lat=lats[:-1]).t2m.values.flatten(),
    values_meas.sel(lat=lats[:-1]).toa_lw_clr_c_mon.values.flatten(),
    c=lc1,
    s=1,
    rasterized=True,
    alpha=0.03
)

latvals = [70, -65, -2, -10, 15]
lonvals = [110, 250, 27, 0, 80]

titles = ['a) ', 'b) ', 'c) ', 'd) ', 'e) ']

style = "Simple, tail_width=0.3, head_width=3, head_length=3"
kw = dict(arrowstyle=style, color="k")

a1 = patches.FancyArrowPatch(
    (284, 221), (259, 171), connectionstyle="arc3,rad=-0.3", **kw)
a2 = patches.FancyArrowPatch(
    (274.5, 229), (271.5, 218), connectionstyle="arc3,rad=-0.3", **kw)
a3 = patches.FancyArrowPatch(
    (297, 272), (297.7, 274.5), connectionstyle="arc3,rad=0.3", **kw)
a4 = patches.FancyArrowPatch(
    (294.9, 294), (298, 294), connectionstyle="arc3,rad=0.4", **kw)
a5 = patches.FancyArrowPatch(
    (301, 284), (297.5, 295), connectionstyle="arc3,rad=-0.4", **kw)

arrs = [a1, a2, a3, a4, a5]

for ind in range(len(latvals)):

    latval = latvals[ind]
    lonval = lonvals[ind]

    i_raw = latval
    j_raw = lonval

    # Bottom loop plots
    axs.plot(
        values_meas.t2m.sel(lat=i_raw, lon=j_raw).values,
        values_meas.toa_lw_clr_c_mon.sel(lat=i_raw, lon=j_raw).values,
        c="k",
        # label="Calculated OLR",
    )
    axs.plot(
        values_meas.t2m.sel(lat=i_raw, lon=j_raw).values[0::11],
        values_meas.toa_lw_clr_c_mon.sel(lat=i_raw, lon=j_raw).values[0::11],
        c="k",
    )
    cplot = axs.scatter(
        values_meas.t2m.sel(lat=i_raw, lon=j_raw).values,
        values_meas.toa_lw_clr_c_mon.sel(lat=i_raw, lon=j_raw).values,
        c=month_list,
        cmap=cmap_new,
        s=30,
    )
    axs.set_title('f$\,$)')

    # Top plots
    ax = top_axs[ind]
    ax.plot(
        values_meas.t2m.sel(lat=i_raw, lon=j_raw).values,
        values_meas.toa_lw_clr_c_mon.sel(lat=i_raw, lon=j_raw).values,
        c="k",
        #   label="Calculated OLR",
    )
    ax.plot(
        values_meas.t2m.sel(lat=i_raw, lon=j_raw).values[0::11],
        values_meas.toa_lw_clr_c_mon.sel(lat=i_raw, lon=j_raw).values[0::11],
        c="k",
    )
    cplot = ax.scatter(
        values_meas.t2m.sel(lat=i_raw, lon=j_raw).values,
        values_meas.toa_lw_clr_c_mon.sel(lat=i_raw, lon=j_raw).values,
        c=month_list,
        cmap=cmap_new,
        s=30,
    )
    ax.set_title(titles[ind]+str(i_raw)+', '+str(j_raw))
    ax.add_patch(arrs[ind])
    ax.margins(x=0.2,y=0.2)

ax3.set_xticks([296.5, 297.5])
    
top_axs[0].set_ylabel(r'OLR (Wm$^{-2}$)')
axs.set_xlabel(r'T$_s$ (K)')
axs.set_ylabel(r'OLR (Wm$^{-2}$)')
fig.suptitle('Latitude, Longitude')

cbar = fig.colorbar(
    cplot,
    ax=axs,
    # label='Month',
    fraction=0.1,
    orientation="vertical",
    aspect=40,
    pad=0,
    shrink=0.8,
    ticks=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    
)

cbar.ax.set_title(r'Month', y=1.02, x=2.5)

plt.savefig('../../Figures/After first review/Fig 2 CERES.pdf', format='pdf',
            bbox_inches='tight', dpi=300)
plt.close()

# Fig. 3

In [8]:
# CERES 
values_meas = xr.open_dataset('../../Data/CERES/clear_sky_ceres.nc')
# Offline calculations from cluster
values_calc = xr.open_dataset('../../Data/Cluster/clear_sky_calculated.nc') 

lats = values_meas.lat.values
lons = values_meas.lon.values

In [ ]:
plotvar = values_meas.hyst.values

lon_long = values_meas.lon.values
plotvar_cyc = np.zeros((len(lats), len(lons)))
plotvar_cyc, lon_long = add_cyclic_point(plotvar_cyc, coord=lon_long)
for i in range(len(lats)):
    for j in range(len(lons)):
        plotvar_cyc[i, j] = plotvar[i, j]
plotvar_cyc[:, len(lons)] = plotvar_cyc[:, 0]

fig = plt.figure(figsize=(6, 5.6), constrained_layout=True)
widths = [1, 1]
heights = [2, 0.9]
gs = fig.add_gridspec(
    ncols=2, nrows=2, width_ratios=widths, height_ratios=heights)

SIZE = 8
plt.rc('font', size=SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=SIZE)     # fontsize of the x and y labels
plt.rc('xtick', labelsize=SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SIZE)    # legend fontsize
plt.rc('figure', titlesize=SIZE)   # fontsize of the figure title

ax1 = fig.add_subplot(gs[0, :], projection=ccrs.PlateCarree())
ax1.coastlines()
C1 = ax1.pcolor(
    lon_long, lats, plotvar_cyc, transform=ccrs.PlateCarree(), cmap='coolwarm', rasterized=True
)
C1.set_clim(vmin=-20, vmax=20)
ax1.set_title('a)')

ax1.set_xticks([-180, -90, 0, 90, 180], crs=ccrs.PlateCarree())
ax1.set_yticks([-90, 0, 90], crs=ccrs.PlateCarree())
lon_formatter = LongitudeFormatter(number_format='.0f',
                                   dateline_direction_label=True)
lat_formatter = LatitudeFormatter(number_format='.0f')
ax1.xaxis.set_major_formatter(lon_formatter)
ax1.yaxis.set_major_formatter(lat_formatter)

# Colourbars
fig.colorbar(
    C1,
    ax=ax1,
    label="OLR Loopiness, $\mathcal{O}$ (Wm$^{-2}$)",
    pad=-0.005,
    aspect=40,
    fraction=0.1,
    # shrink=0.97,
    orientation="horizontal",
)

xvals_l = values_meas.mean(
    dim='month').toa_lw_clr_c_mon.values.flatten()
yvals_l = values_calc.mean(dim='month').olr_calc.values.flatten()

xvals_r = values_meas.mean(
    dim='month').hyst.values.flatten()
yvals_r = values_calc.mean(dim='month').hyst.values.flatten()


# Bottom left plot 
ax2 = fig.add_subplot(gs[1, 0])

hist1 = ax2.hist2d(xvals_l, yvals_l, 100, cmap='Greys',
                   norm=LogNorm(), vmin=0.2)
ax2.plot([np.amin(xvals_l), np.amax(xvals_l)], [np.amin(xvals_l), np.amax(
    xvals_l)], c='C1', label='1:1 Line', linestyle='--', linewidth=2, rasterized=True)

ax2.set_xlabel(r'CERES (Wm$^{-2}$)')
ax2.set_ylabel(r'Offline Calculations (Wm$^{-2}$)')
ax2.set_title('b) Annual Mean OLR')
ax2.legend()

cbar1 = fig.colorbar(hist1[-1], ax=ax2, fraction=0, pad=-0.005, shrink=0.7, ticks=[1000, 100, 10, 1])
cbar1.ax.set_title(r'$\frac{\#}{(W m^{-2})^2}$', fontsize=10, y=1.15, x=4)


# Bottom right plot
ax3 = fig.add_subplot(gs[1, 1])

ax3.hist2d(xvals_r, yvals_r, 100, cmap='Greys', norm=LogNorm(), vmin=0.2)
ax3.plot([np.amin(xvals_r), np.amax(xvals_r)], [np.amin(xvals_r), np.amax(
    xvals_r)], c='C1', label='1:1 Line', linestyle='--', linewidth=2, rasterized=True)

ax3.set_xlabel(r'CERES (Wm$^{-2}$)')
ax3.set_ylabel(r'Offline Calculations (Wm$^{-2}$)')
ax3.set_title("c) OLR Loopiness, $\mathcal{O}$")
ax3.legend()

cbar2 = fig.colorbar(hist1[-1], ax=ax3, fraction=0, pad=-0.005, shrink=0.7, ticks=[1000, 100, 10, 1])
cbar2.ax.set_title(r'$\frac{\#}{(W m^{-2})^2}$', fontsize=10, y=1.15, x=4)

plt.savefig('../../Figures/After first review/Fig 3 CERES.pdf',
            format='pdf', bbox_inches='tight', dpi=300)
plt.close()

### Mean Absolute Error

In [16]:
xvals_l = values_meas.mean(
    dim='month').toa_lw_clr_c_mon.values.flatten()
yvals_l = values_calc.mean(dim='month').olr_calc.values.flatten()

xvals_r = values_meas.mean(
    dim='month').hyst.values.flatten()
yvals_r = values_calc.mean(dim='month').hyst.values.flatten()

diff_l = []
diff_r = []

for i in range(len(xvals_l)):
    diff_l.append(np.abs(yvals_l[i]-xvals_l[i]))
    diff_r.append(np.abs(yvals_r[i]-xvals_r[i]))

mae_l = np.mean(diff_l)
mae_r = np.mean(diff_r)

print('MAE Annual Mean:', mae_l)
print('MAE Loopiness :', mae_r)

### Calculating the Bias 

In [17]:
# Calculate bias of cluster - CERES
bias = values_calc.hyst - values_meas.hyst
bias.mean(dim=('lat','lon')).values[()]

1.1663813066790574

# Fig. 4

In [68]:
values_meas = xr.open_dataset('../../Data/CERES/clear_sky_ceres.nc')
values_meas_base = xr.open_dataset('../../Data/Cluster/Combined_data_ceres_base.nc')
values_meas_const_r = xr.open_dataset('../../Data/Cluster/clear_sky_calculated_const_rh.nc')
values_meas_const_t = xr.open_dataset('../../Data/Cluster/clear_sky_calculated_const_t.nc')

values_meas_atm = xr.open_dataset('../../Data/Cluster/data_atm.nc')

lats = values_meas.lat.values
lons = values_meas.lon.values
levels = values_meas_atm.level.values

In [ ]:
month_val = 0
month_list = np.arange(1, 13, 1)

fig, axs = plt.subplots(nrows=4, ncols=3, figsize=(
    6, 6), constrained_layout=True)

# Location, off of the coast of California (latitude value, longitude value)
lav = 32
lov = 237

month_list = np.arange(1, 13)
line_colour = "#7d7d7d"

SIZE = 8
plt.rc('font', size=SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=SIZE)     # fontsize of the x and y labels
plt.rc('xtick', labelsize=SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SIZE)    # legend fontsize
plt.rc('figure', titlesize=SIZE)   # fontsize of the figure title

lcmap1 = mpl.cm.get_cmap('Blues')
lcmap2 = mpl.cm.get_cmap('Oranges')
lcmap3 = mpl.cm.get_cmap('Greens')

mean_t2m = np.mean(values_meas.sel(lat=lav, lon=lov).t2m.values)

for i in range(12):

    lc1 = lcmap1(i/11)
    lc2 = lcmap2(i/11)

    a_val = 1  # * i/11

    axs[0, 0].plot(
        [j + (values_meas.sel(lat=lav, lon=lov).t2m.values[i] - mean_t2m)
         for j in values_meas_atm.mean(dim="month").t.values],
        levels,
        c=lc1,
        alpha=a_val,
    )

    axs[1, 0].plot(
        values_meas_atm.sel(month=month_list[i]).t.values,
        levels,
        c=lc1,
        alpha=a_val,
    )

    axs[2, 0].plot(
        [j + (values_meas.sel(lat=lav, lon=lov).t2m.values[i] - mean_t2m)
         for j in values_meas_atm.mean(dim="month").t.values],
        levels,
        c=lc1,
        alpha=a_val,
    )

    axs[3, 0].plot(
        values_meas_atm.sel(month=month_list[i]).t.values,
        levels,
        c=lc1,
        alpha=a_val,
    )

    axs[2, 1].plot(
        values_meas_atm.sel(month=month_list[i]).r.values,
        levels,
        c=lc2,
        alpha=a_val,
    )

    axs[3, 1].plot(
        values_meas_atm.sel(month=month_list[i]).r.values,
        levels,
        c=lc2,
        alpha=a_val,
    )

# Top two RH
axs[0, 1].plot(
    values_meas_atm.mean(dim="month").r.values,
    levels,
    c=lcmap2(1.0),
)

axs[1, 1].plot(
    values_meas_atm.mean(dim="month").r.values,
    levels,
    c=lcmap2(1.0),
)

# Base Case
axs[0, 2].plot(
    values_meas_base.sel(lat=lav, lon=lov).ts.values,
    values_meas_base.sel(lat=lav, lon=lov).olr_calc.values,
    c=line_colour
)
axs[0, 2].plot(
    values_meas_base.sel(lat=lav, lon=lov).ts.values[0::11],
    values_meas_base.sel(lat=lav, lon=lov).olr_calc.values[0::11],
    c=line_colour
)
axs[0, 2].scatter(
    values_meas_base.sel(lat=lav, lon=lov).ts.values,
    values_meas_base.sel(lat=lav, lon=lov).olr_calc.values,
    c=month_list,
    cmap=lcmap3,
    s=30,
)
axs[0, 2].margins(x=0.2,y=0.2)

# Temperature Variation Only Case
axs[1, 2].plot(
    values_meas_const_r.sel(lat=lav, lon=lov).ts.values,
    values_meas_const_r.sel(lat=lav, lon=lov).olr_calc.values,
    c=line_colour
)
axs[1, 2].plot(
    values_meas_const_r.sel(lat=lav, lon=lov).ts.values[0::11],
    values_meas_const_r.sel(lat=lav, lon=lov).olr_calc.values[0::11],
    c=line_colour
)
axs[1, 2].scatter(
    values_meas_const_r.sel(lat=lav, lon=lov).ts.values,
    values_meas_const_r.sel(lat=lav, lon=lov).olr_calc.values,
    c=month_list,
    cmap=lcmap3,
    s=30,
)
axs[1, 2].margins(x=0.2,y=0.2)

# Moisture Variation Only Case
axs[2, 2].plot(
    values_meas_const_t.sel(lat=lav, lon=lov).ts.values,
    values_meas_const_t.sel(lat=lav, lon=lov).olr_calc.values,
    c=line_colour
)
axs[2, 2].plot(
    values_meas_const_t.sel(lat=lav, lon=lov).ts.values[0::11],
    values_meas_const_t.sel(lat=lav, lon=lov).olr_calc.values[0::11],
    c=line_colour
)
axs[2, 2].scatter(
    values_meas_const_t.sel(lat=lav, lon=lov).ts.values,
    values_meas_const_t.sel(lat=lav, lon=lov).olr_calc.values,
    c=month_list,
    cmap=lcmap3,
    s=30,
)
axs[2, 2].margins(x=0.2,y=0.2)

# Full Case
axs[3, 2].plot(
    values_meas.sel(lat=lav, lon=lov).t2m.values,
    values_meas.sel(lat=lav, lon=lov).toa_lw_clr_c_mon.values,
    c=line_colour
)
axs[3, 2].plot(
    values_meas.sel(lat=lav, lon=lov).t2m.values[0::11],
    values_meas.sel(lat=lav, lon=lov).toa_lw_clr_c_mon.values[0::11],
    c=line_colour
)
axs[3, 2].scatter(
    values_meas.sel(lat=lav, lon=lov).t2m.values,
    values_meas.sel(lat=lav, lon=lov).toa_lw_clr_c_mon.values,
    c=month_list,
    cmap=lcmap3,
    s=30,
)
axs[3, 2].margins(x=0.2,y=0.2)

# Formatting 
for i1 in range(3):
    for i2 in range(4):
        if i1 == 0:
            axs[i2, i1].set_ylabel("Pressure (mBar)")

axs[0, 0].invert_yaxis()
axs[1, 0].invert_yaxis()
axs[2, 0].invert_yaxis()
axs[3, 0].invert_yaxis()
axs[0, 1].invert_yaxis()
axs[1, 1].invert_yaxis()
axs[2, 1].invert_yaxis()
axs[3, 1].invert_yaxis()

labels = ['']

axs[0,1].set_yticklabels(labels)
axs[1,1].set_yticklabels(labels)
axs[2,1].set_yticklabels(labels)
axs[3,1].set_yticklabels(labels)

axs[0,0].set_xticklabels(labels)
axs[1,0].set_xticklabels(labels)
axs[2,0].set_xticklabels(labels)
axs[0,1].set_xticklabels(labels)
axs[1,1].set_xticklabels(labels)
axs[2,1].set_xticklabels(labels)
axs[0,2].set_xticklabels(labels)
axs[1,2].set_xticklabels(labels)
axs[2,2].set_xticklabels(labels)

temp_ticks = [200, 230, 260, 290]
axs[0,0].set_xticks(temp_ticks)
axs[1,0].set_xticks(temp_ticks)
axs[2,0].set_xticks(temp_ticks)
axs[3,0].set_xticks(temp_ticks)

rh_ticks = [0, 30, 60, 90]
axs[0,1].set_xticks(rh_ticks)
axs[1,1].set_xticks(rh_ticks)
axs[2,1].set_xticks(rh_ticks)
axs[3,1].set_xticks(rh_ticks)

axs[0,2].yaxis.tick_right()
axs[1,2].yaxis.tick_right()
axs[2,2].yaxis.tick_right()
axs[3,2].yaxis.tick_right()
axs[0,2].yaxis.set_label_position("right")
axs[1,2].yaxis.set_label_position("right")
axs[2,2].yaxis.set_label_position("right")
axs[3,2].yaxis.set_label_position("right")

temp_xlim = axs[1,0].get_xlim()
axs[0,0].set_xlim(temp_xlim)
axs[1,0].set_xlim(temp_xlim)
axs[2,0].set_xlim(temp_xlim)
axs[3,0].set_xlim(temp_xlim)

rh_xlim = axs[2,1].get_xlim()
axs[0,1].set_xlim(rh_xlim)
axs[1,1].set_xlim(rh_xlim)
axs[2,1].set_xlim(rh_xlim)
axs[3,1].set_xlim(rh_xlim)

axs[0,2].set_ylabel('OLR (Wm$^{-2}$)', rotation=-90, labelpad=15)
axs[1,2].set_ylabel('OLR (Wm$^{-2}$)', rotation=-90, labelpad=15)
axs[2,2].set_ylabel('OLR (Wm$^{-2}$)', rotation=-90, labelpad=15)
axs[3,2].set_ylabel('OLR (Wm$^{-2}$)', rotation=-90, labelpad=15)

axs[3, 0].set_xlabel("Temperature (K)")
axs[3, 1].set_xlabel("Relative Humidity (%)")
axs[3, 2].set_xlabel('T$_s$ (K)')

xv = 1.55
yv = 1.08
fs = 8

axs[0,1].set_title('Base Case (Zero Loopiness)')
axs[1,1].set_title('Seasonal Temperature Variation Only')
axs[2,1].set_title('Seasonal Moisture Variation Only')
axs[3,1].set_title('Full Case (Full Loopiness)')

axs[0, 0].arrow(250, 550, 2.5, 0, head_width=40, head_length=2.5, fc='k')
axs[0, 0].arrow(250, 550, -2.5, 0, head_width=40, head_length=2.5, fc='k')

axs[2, 0].arrow(250, 550, 2.5, 0, head_width=40, head_length=2.5, fc='k')
axs[2, 0].arrow(250, 550, -2.5, 0, head_width=40, head_length=2.5, fc='k')

axs[1, 0].arrow(220, 350, 2.5, 0, head_width=40, head_length=2.5, fc='k')
axs[1, 0].arrow(220, 350, -2.5, 0, head_width=40, head_length=2.5, fc='k')

axs[1, 0].arrow(265, 850, 7.5, 0, head_width=40, head_length=2.5, fc='k')
axs[1, 0].arrow(265, 850, -7.5, 0, head_width=40, head_length=2.5, fc='k')

axs[3, 0].arrow(220, 350, 2.5, 0, head_width=40, head_length=2.5, fc='k')
axs[3, 0].arrow(220, 350, -2.5, 0, head_width=40, head_length=2.5, fc='k')

axs[3, 0].arrow(265, 850, 7.5, 0, head_width=40, head_length=2.5, fc='k')
axs[3, 0].arrow(265, 850, -7.5, 0, head_width=40, head_length=2.5, fc='k')

axs[2, 1].arrow(72, 830, 2.5, 0, head_width=40, head_length=2.5, fc='k')
axs[2, 1].arrow(72, 830, -2.5, 0, head_width=40, head_length=2.5, fc='k')

axs[2, 1].arrow(72, 250, 7.5, 0, head_width=40, head_length=2.5, fc='k')
axs[2, 1].arrow(72, 250, -7.5, 0, head_width=40, head_length=2.5, fc='k')

axs[3, 1].arrow(72, 830, 2.5, 0, head_width=40, head_length=2.5, fc='k')
axs[3, 1].arrow(72, 830, -2.5, 0, head_width=40, head_length=2.5, fc='k')

axs[3, 1].arrow(72, 250, 7.5, 0, head_width=40, head_length=2.5, fc='k')
axs[3, 1].arrow(72, 250, -7.5, 0, head_width=40, head_length=2.5, fc='k')

plt.savefig('../../Figures/After first review/Fig 4 CERES.pdf',
            bbox_inches='tight', format='pdf')  # Save the figure
plt.close()

# Fig. 5

In [80]:
values_t = xr.open_dataset('../../Data/Cluster/clear_sky_calculated_const_t.nc')
values_r = xr.open_dataset('../../Data/Cluster/clear_sky_calculated_const_rh.nc')

lats = values_t.lat.values
lons = values_t.lon.values

In [ ]:
# Variables that you want to plot
plotvar1 = values_r.hyst.values
plotvar2 = values_t.hyst.values

# Adding a cyclic point to the two variables
# This removes a white line at lon = 0
lon_long = values_r.lon.values
plotvar_cyc1 = np.zeros((len(lats), len(lons)))
plotvar_cyc1, lon_long = add_cyclic_point(plotvar_cyc1, coord=lon_long)
for i in range(len(lats)):
    for j in range(len(lons)):
        plotvar_cyc1[i, j] = plotvar1[i, j]
plotvar_cyc1[:, len(lons)] = plotvar_cyc1[:, 0]

lon_long = values_r.lon.values
plotvar_cyc2 = np.zeros((len(lats), len(lons)))
plotvar_cyc2, lon_long = add_cyclic_point(plotvar_cyc2, coord=lon_long)
for i in range(len(lats)):
    for j in range(len(lons)):
        plotvar_cyc2[i, j] = plotvar2[i, j]
plotvar_cyc2[:, len(lons)] = plotvar_cyc2[:, 0]

# Plotting
fig = plt.figure(figsize=(6, 2.4), constrained_layout=True)
gs = fig.add_gridspec(ncols=2, nrows=1)

SIZE = 8
plt.rc('font', size=SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=SIZE)     # fontsize of the x and y labels
plt.rc('xtick', labelsize=SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SIZE)    # legend fontsize
plt.rc('figure', titlesize=SIZE)   # fontsize of the figure title

# Left map
ax1 = fig.add_subplot(gs[0], projection=ccrs.PlateCarree())
ax1.coastlines()
C1 = ax1.pcolor(
    lon_long, lats, plotvar_cyc1, transform=ccrs.PlateCarree(), cmap='coolwarm', rasterized=True
)
C1.set_clim(vmin=-20, vmax=20)
ax1.set_title("a) Temperature Variation Only")

ax1.set_xticks([-180, -90, 0, 90, 180], crs=ccrs.PlateCarree())
ax1.set_yticks([-90, 0, 90], crs=ccrs.PlateCarree())
lon_formatter = LongitudeFormatter(number_format='.0f',
                                   dateline_direction_label=True)
lat_formatter = LatitudeFormatter(number_format='.0f')
ax1.xaxis.set_major_formatter(lon_formatter)
ax1.yaxis.set_major_formatter(lat_formatter)

# Colourbars
fig.colorbar(
    C1,
    ax=ax1,
    label="OLR Loopiness, $\mathcal{O}$ (Wm$^{-2}$)",
    pad=0,
    aspect=20,
    fraction=0.1,
    # shrink=0.95,
    orientation="horizontal",
)

# Right map
ax2 = fig.add_subplot(gs[1], projection=ccrs.PlateCarree())
ax2.coastlines()
C2 = ax2.pcolor(
    lon_long, lats, plotvar_cyc2, transform=ccrs.PlateCarree(), cmap='coolwarm', rasterized=True
)
C2.set_clim(vmin=-20, vmax=20)
ax2.set_title("b) Moisture Variation Only")

ax2.set_xticks([-180, -90, 0, 90, 180], crs=ccrs.PlateCarree())
ax2.set_yticks([-90, 0, 90], crs=ccrs.PlateCarree())
lon_formatter = LongitudeFormatter(number_format='.0f',
                                   dateline_direction_label=True)
lat_formatter = LatitudeFormatter(number_format='.0f')
ax2.xaxis.set_major_formatter(lon_formatter)
ax2.yaxis.set_major_formatter(lat_formatter)

# Colourbars
fig.colorbar(
    C2,
    ax=ax2,
    label="OLR Loopiness, $\mathcal{O}$ (Wm$^{-2}$)",
    pad=0,
    aspect=20,
    fraction=0.1,
    # shrink=0.95,
    orientation="horizontal",
)

plt.savefig('../../Figures/After first review/Fig 5 CERES.pdf',
            format='pdf', dpi=300, bbox_inches='tight')
plt.close()

# Fig. 6

In [84]:
values_as = xr.open_dataset('../../Data/CERES/all_sky_ceres.nc')
values_cs = xr.open_dataset('../../Data/CERES/clear_sky_ceres.nc')

lats = values_cs.lat.values
lons = values_cs.lon.values

In [ ]:
# Variables that you want to plot
plotvar1 = values_cs.hyst_over_olr_range.values
plotvar2 = values_as.hyst_over_olr_range.values

# Adding a cyclic point to the two variables
# This removes a white line at lon = 0
lon_long = values_cs.lon.values
plotvar_cyc1 = np.zeros((len(lats), len(lons)))
plotvar_cyc1, lon_long = add_cyclic_point(plotvar_cyc1, coord=lon_long)
for i in range(len(lats)):
    for j in range(len(lons)):
        plotvar_cyc1[i, j] = plotvar1[i, j]
plotvar_cyc1[:, len(lons)] = plotvar_cyc1[:, 0]

lon_long = values_cs.lon.values
plotvar_cyc2 = np.zeros((len(lats), len(lons)))
plotvar_cyc2, lon_long = add_cyclic_point(plotvar_cyc2, coord=lon_long)
for i in range(len(lats)):
    for j in range(len(lons)):
        plotvar_cyc2[i, j] = plotvar2[i, j]
plotvar_cyc2[:, len(lons)] = plotvar_cyc2[:, 0]

# Plotting
fig = plt.figure(figsize=(6, 2.4), constrained_layout=True)
gs = fig.add_gridspec(ncols=2, nrows=1)

SIZE = 8
plt.rc('font', size=SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=SIZE)     # fontsize of the x and y labels
plt.rc('xtick', labelsize=SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SIZE)    # legend fontsize
plt.rc('figure', titlesize=SIZE)   # fontsize of the figure title

# Left map
ax1 = fig.add_subplot(gs[0], projection=ccrs.PlateCarree())
ax1.coastlines()
C1 = ax1.pcolor(
    lon_long, lats, plotvar_cyc1, transform=ccrs.PlateCarree(), cmap='coolwarm', rasterized=True
)
C1.set_clim(vmin=-100, vmax=100)
ax1.set_title("a) Clear Sky")

ax1.set_xticks([-180, -90, 0, 90, 180], crs=ccrs.PlateCarree())
ax1.set_yticks([-90, 0, 90], crs=ccrs.PlateCarree())
lon_formatter = LongitudeFormatter(number_format='.0f',
                                   dateline_direction_label=True)
lat_formatter = LatitudeFormatter(number_format='.0f')
ax1.xaxis.set_major_formatter(lon_formatter)
ax1.yaxis.set_major_formatter(lat_formatter)

# Colourbars
cbar1 = fig.colorbar(
    C1,
    ax=ax1,
    label=" $\mathcal{O}$ / OLR Range (%)",
    pad=0,
    aspect=20,
    fraction=0.1,
    # shrink=0.95,
    orientation="horizontal",
)

cbar1.ax.set_xticklabels(
    ['-100%', '-50%', '0%', '50%', '100%'])

# Right map
ax2 = fig.add_subplot(gs[1], projection=ccrs.PlateCarree())
ax2.coastlines()
C2 = ax2.pcolor(
    lon_long, lats, plotvar_cyc2, transform=ccrs.PlateCarree(), cmap='coolwarm', rasterized=True
)
C2.set_clim(vmin=-100, vmax=100)
ax2.set_title("b) All Sky")

ax2.set_xticks([-180, -90, 0, 90, 180], crs=ccrs.PlateCarree())
ax2.set_yticks([-90, 0, 90], crs=ccrs.PlateCarree())
lon_formatter = LongitudeFormatter(number_format='.0f',
                                   dateline_direction_label=True)
lat_formatter = LatitudeFormatter(number_format='.0f')
ax2.xaxis.set_major_formatter(lon_formatter)
ax2.yaxis.set_major_formatter(lat_formatter)

# Colourbars
cbar2 = fig.colorbar(
    C2,
    ax=ax2,
    label="$\mathcal{O}$ / OLR Range (%)",
    pad=0,
    aspect=20,
    fraction=0.1,
    # shrink=0.95,
    orientation="horizontal",
)

cbar2.ax.set_xticklabels(
    ['-100%', '-50%', '0%', '50%', '100%'])

plt.savefig('../../Figures/After first review/Fig 6 CERES.pdf',
            format='pdf', dpi=300, bbox_inches='tight')
plt.close()

### Global normalised values

In [40]:
area_factors = np.zeros_like(values_cs.mean(dim='month').t2m.values)

lats = values_cs.lat.values
lons = values_cs.lon.values

for i in range(len(lats)):
    area_factors[i, :] = np.cos(lats[i]*2*np.pi/360)

values_cs['area_factors'] = (('lat', 'lon'), area_factors)

hyst_over_olr_area_scaled_cs = np.zeros_like(values_cs.mean(dim='month').t2m.values)
hyst_over_olr_area_scaled_as = np.zeros_like(values_as.mean(dim='month').t2m.values)

for i in range(len(lats)):
    for j in range(len(lons)):
        hyst_over_olr_area_scaled_cs[i,j] = values_cs.sel(lat=lats[i], lon=lons[j]).area_factors.values[()] * np.abs(values_cs.sel(lat=lats[i], lon=lons[j]).hyst_over_olr_range.values[()])
        hyst_over_olr_area_scaled_as[i,j] = values_cs.sel(lat=lats[i], lon=lons[j]).area_factors.values[()] * np.abs(values_as.sel(lat=lats[i], lon=lons[j]).hyst_over_olr_range.values[()])

values_cs['hyst_over_olr_area_scaled'] = (('lat', 'lon'), hyst_over_olr_area_scaled_cs)
values_as['hyst_over_olr_area_scaled'] = (('lat', 'lon'), hyst_over_olr_area_scaled_as)

glbl_cs = np.sum(values_cs.hyst_over_olr_area_scaled.values.flatten()) / np.sum(values_cs.area_factors.values.flatten())
glbl_as = np.sum(values_as.hyst_over_olr_area_scaled.values.flatten()) / np.sum(values_cs.area_factors.values.flatten())

print('Global area weighted clear sky mean is', '%.2f' % glbl_cs, '%')
print('Global area weighted all sky mean is', '%.2f' % glbl_as, '%')

# Supplementary Information 

In [4]:
# ERA5
values_meas_era5 = xr.open_dataset('../../Data/Other/values_meas_dir_int_hyst.nc')

# CERES
values_meas_ceres = xr.open_dataset('../../Data/CERES/clear_sky_ceres.nc')

lats = values_meas_ceres.lat.values
lons = values_meas_ceres.lon.values

In [ ]:
plotvar1 = values_meas_era5.directional_int_hyst.values
plotvar2 = values_meas_ceres.hyst.values

lon_long1 = values_meas_era5.lon.values
plotvar_cyc1 = np.zeros((len(lats), len(lons)))
plotvar_cyc1, lon_long1 = add_cyclic_point(plotvar_cyc1, coord=lon_long1)
for i in range(len(lats)):
    for j in range(len(lons)):
        plotvar_cyc1[i, j] = plotvar1[i, j]
plotvar_cyc1[:, len(lons)] = plotvar_cyc1[:, 0]

lon_long2 = values_meas_ceres.lon.values
plotvar_cyc2 = np.zeros((len(lats), len(lons)))
plotvar_cyc2, lon_long2 = add_cyclic_point(plotvar_cyc2, coord=lon_long2)
for i in range(len(lats)):
    for j in range(len(lons)):
        plotvar_cyc2[i, j] = plotvar2[i, j]
plotvar_cyc2[:, len(lons)] = plotvar_cyc2[:, 0]

fig = plt.figure(figsize=(6, 7), constrained_layout=True)
gs = fig.add_gridspec(
    ncols=1, nrows=2)

SIZE = 8
plt.rc('font', size=SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=SIZE)     # fontsize of the x and y labels
plt.rc('xtick', labelsize=SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SIZE)    # legend fontsize
plt.rc('figure', titlesize=SIZE)   # fontsize of the figure title

ax1 = fig.add_subplot(gs[0], projection=ccrs.PlateCarree())
ax1.coastlines()
C1 = ax1.pcolor(
    lon_long1, lats, plotvar_cyc1, transform=ccrs.PlateCarree(), cmap='coolwarm', rasterized=True
)
C1.set_clim(vmin=-20, vmax=20)
ax1.set_title('a) ERA5')

ax1.set_xticks([-180, -90, 0, 90, 180], crs=ccrs.PlateCarree())
ax1.set_yticks([-90, 0, 90], crs=ccrs.PlateCarree())
lon_formatter = LongitudeFormatter(number_format='.0f',
                                   dateline_direction_label=True)
lat_formatter = LatitudeFormatter(number_format='.0f')
ax1.xaxis.set_major_formatter(lon_formatter)
ax1.yaxis.set_major_formatter(lat_formatter)

ax2 = fig.add_subplot(gs[1], projection=ccrs.PlateCarree())
ax2.coastlines()
C2 = ax2.pcolor(
    lon_long2, lats, plotvar_cyc2, transform=ccrs.PlateCarree(), cmap='coolwarm', rasterized=True
)
C2.set_clim(vmin=-20, vmax=20)
ax2.set_title('b) CERES')

ax2.set_xticks([-180, -90, 0, 90, 180], crs=ccrs.PlateCarree())
ax2.set_yticks([-90, 0, 90], crs=ccrs.PlateCarree())
lon_formatter = LongitudeFormatter(number_format='.0f',
                                   dateline_direction_label=True)
lat_formatter = LatitudeFormatter(number_format='.0f')
ax2.xaxis.set_major_formatter(lon_formatter)
ax2.yaxis.set_major_formatter(lat_formatter)


# Colourbar
fig.colorbar(
    C2,
    ax=ax2,
    label="OLR Loopiness, $\mathcal{O}$ (Wm$^{-2}$)",
    pad=-0.005,
    aspect=40,
    fraction=0.1,
    # shrink=0.97,
    orientation="horizontal",
)

plt.savefig('../../Figures/After first review/SI.pdf',
            format='pdf', bbox_inches='tight', dpi=300)
plt.close()